In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
import spacy

from typing import Union
from collections import Counter
import sys

2023-07-08 08:44:29.719980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_parquet("/Users/luis.morales/Desktop/arxiv-paper-recommender/data/processed/reduced_arxiv_papers.parquet.gzip").sample(frac=None)
df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cleaned_abstracts,len_abstract
908956,2301.08674,Sergio Pezzini,"Daniel Vaquero, Vito Cleric\`o, Michael Schmit...",Phonon-mediated room-temperature quantum Hall ...,"17 pages, 4 figures. Supplementary information...","Nature Communications 14, 318 (2023)",10.1038/s41467-023-35986-3,None,cond-mat.mes-hall,http://arxiv.org/licenses/nonexclusive-distrib...,The quantum Hall (QH) effect in two-dimensio...,"[{'created': 'Fri, 20 Jan 2023 16:53:31 GMT', ...",2023-01-23,"[[Vaquero, Daniel, ], [Clericò, Vito, ], [Schm...",quantum hall qh effect dimensional electron...,842


In [3]:
df = pd.read_parquet("/Users/luis.morales/Desktop/arxiv-paper-recommender/data/processed/reduced_arxiv_papers.parquet.gzip").sample(30000).reset_index(drop=True)

In [4]:
df.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,cleaned_abstracts,len_abstract
0,cond-mat/0509555,Ricardo Brito,M.H.Ernst and R. Brito,Generalized Green-Kubo formulas for fluids wit...,"14 pages, no figures. Version 2: expanded Intr...",Phys. Rev. E 72 (2006) 061102,10.1103/PhysRevE.72.061102,None,cond-mat.stat-mech,None,We present a generalization of the Green-Kub...,"[{'created': 'Wed, 21 Sep 2005 17:18:42 GMT', ...",2007-05-23,"[[Ernst, M. H., ], [Brito, R., ]]",present generalization green kubo expressio...,814
1,cond-mat/0204489,Jizhong Lou,"Jizhong Lou, Shaojin Qin, Changfeng Chen, Zhao...",Field-induced gap in the spin-1/2 antiferromag...,"8 pages, 6 figures","Phys. Rev. B 65, 064420 (2002)",10.1103/PhysRevB.65.064420,None,cond-mat.str-el,None,We study the spin-1/2 antiferromagnetic Heis...,"[{'created': 'Tue, 23 Apr 2002 14:28:46 GMT', ...",2009-11-07,"[[Lou, Jizhong, ], [Qin, Shaojin, ], [Chen, Ch...",study spin-1/2 antiferromagnetic heisenberg...,856
2,0708.0723,Salvatore Capozziello,"S. Capozziello, A. Stabile, A. Troisi",The Newtonian Limit of F(R) gravity,16 pages,"Phys.Rev.D76:104019,2007",10.1103/PhysRevD.76.104019,None,gr-qc astro-ph,None,A general analytic procedure is developed to...,"[{'created': 'Mon, 6 Aug 2007 08:46:02 GMT', '...",2008-11-26,"[[Capozziello, S., ], [Stabile, A., ], [Troisi...",general analytic procedure develop deal new...,821
3,1312.3113,Dmitry Shcherbakov M.Sc.,"Dmitry Shcherbakov, Matthias Ehrhardt, Michael...",Force-Gradient Nested Multirate Methods for Ha...,"16 pages 3 figures, submitted to Applied Numer...",None,None,None,math.NA math-ph math.MP quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Force-gradient decomposition methods are use...,"[{'created': 'Wed, 11 Dec 2013 11:08:03 GMT', ...",2013-12-12,"[[Shcherbakov, Dmitry, ], [Ehrhardt, Matthias,...",force gradient decomposition method improve...,535
4,1607.01234,Yadao Solanke,"Y. S. Solanke, D. D.Pawar and V. J. Dagwal",Role of the constant deceleration parameter in...,None,None,None,None,gr-qc,http://arxiv.org/licenses/nonexclusive-distrib...,The main purpose of the present paper is to ...,"[{'created': 'Tue, 5 Jul 2016 12:58:00 GMT', '...",2020-03-06,"[[Solanke, Y. S., ], [Pawar, D. D., ], [Dagwal...",main purpose present paper investigate lrs ...,557


### Read Tokens

In [5]:
def read_list_from_file(file_path: str):
    """
    Reads a text file containing a list and converts it back to a Python list
    Args:
        file_path (str): Path to the text file
    Returns:
        list: The Python list read from the file.
    """
    try:
        with open(file_path, 'r') as file:
            # Read the contents of the file
            file_contents = file.read()
            lines = file_contents.split('\n')
            plist = [item for item in lines if item]

            return plist
    except FileNotFoundError:
        raise FileNotFoundError("The specified file cannot be found.")

In [6]:
# tokens_path = "../data/processed/corpus_tokens.txt"
# tokens = read_list_from_file(tokens_path)

In [7]:
# tokens_set = set(tokens)

In [8]:
# len(tokens_set)

In [9]:
# count_unique_words = Counter(tokens)

In [10]:
# most_common_words = count_unique_words.most_common(int(len(tokens_set) * 0.7))

In [11]:
# dict_of_words = [pair[0] for pair in most_common_words]

In [12]:
# count_unique_words.most_common()

### Continue

In [4]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(df['cleaned_abstracts'])

In [5]:
tfidf_features =  [i for i in vectorizer.get_feature_names_out()]

In [6]:
tfidf_vectors = vectors.toarray()

In [7]:
tfidf_df = pd.DataFrame(tfidf_vectors, columns=tfidf_features)
# 

In [8]:
tfidf_df

,00,000,000001,000002,00002,000025,00007,00009,0001,0001106,...,zy,zygmund,zygoma,zygomatic,zygote,zykin,zytkow,zz,zzgamma,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
full_df = pd.concat([df, tfidf_df], axis=1)

In [10]:
full_df

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,...,zy,zygmund,zygoma,zygomatic,zygote,zykin,zytkow,zz,zzgamma,zzz
0,1109.5512,Keita Owari,Keita Owari,On Admissible Strategies in Robust Utility Max...,Forthcoming in Mathematics and Financial Econo...,"Mathematics and Financial Economics, Vol. 6, N...",10.1007/s11579-012-0068-3,None,q-fin.PM math.PR,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,cs/0403015,Ichiro Adachi,"Ichiro Adachi, Taisuke Hibino, Luc Hinz, Ryosu...",Belle Computing System,"6 pages, 6 figures, talk given at ACAT03, Tsuk...",Nucl.Instrum.Meth. A534 (2004) 53-58,10.1016/j.nima.2004.07.058,None,cs.DC,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1706.05858,Zheng Xie,"Zheng Xie, Miao Li, Jianping Li, Xiaojun Duan,...",Feature analysis of multidisciplinary scientif...,None,"Xie, Z., Li, M., Li, J., Duan, X., & Ouyang, Z...",10.1140/epjds/s13688-018-0134-z,None,physics.soc-ph cs.DL,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1605.09027,Roland Duduchava Prof.,"Tengiz Buchukuri, Roland Duduchava, George Tep...",Laplace-Beltrami equation on hypersurfaces and...,"38 pages, 2 figures",None,None,None,math-ph math.MP,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1110.4381,Jian-Xin Zhu Dr.,"Jian-Xin Zhu, Jean-Pierre Julien, Y. Dubi, A. ...",Local Electronic Structure and Fano Interferen...,"4+ pages, 3 eps figures, revised version with ...",None,10.1103/PhysRevLett.108.186401,LA-UR 11-05833,cond-mat.str-el,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1411.7041,Xi Dong,"Ahmed Almheiri, Xi Dong, Daniel Harlow",Bulk Locality and Quantum Error Correction in ...,"30 pages plus appendices, 10 figures. v2: sect...","JHEP 1504:163,2015",10.1007/JHEP04(2015)163,SU-ITP-14/30,hep-th gr-qc quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,1408.4854,Vitaly Vanchurin,Vitaly Vanchurin,The Continuum of Discrete Trajectories in Eter...,12 pages,"Phys.Rev. D91 (2015) no.2, 02351",10.1103/PhysRevD.91.023511,None,hep-th gr-qc hep-ph,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29997,1008.2215,Andrey Katz,"Daniel Green, Andrey Katz, and Zohar Komargodski",Direct Gaugino Mediation,"26 pages, 1 figure. v2: minor corrections, ref...","Phys.Rev.Lett.106:061801,2011",10.1103/PhysRevLett.106.061801,UMD-PP-10-013,hep-th hep-ph,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29998,1605.05516,Gabriel Riviere,"Nguyen Viet Dang (ICJ), Gabriel Riviere (LPP)",Spectral analysis of morse-smale gradient flows,"Shortened version (56 p.), to appear in Annale...",None,None,None,math.DS math.GT math.SP,http://arxiv.org/licenses/nonexclusive-distrib...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
def preprocessAbstracts(doc: Union[pd.Series, str], nlp = spacy.load('en_core_web_sm')):
    """
    TODO: Optimize NLP Object to only obtain stopwords, lemmas, and tokenize docs.
    
    Cleans and processes the input documents by performing various text cleaning operations.

    Args:
        doc (pd.Series): The documents to be cleaned, passed in a Series object.
        stemming (bool, optional): Specifies whether stemming should be applied. Defaults to False.

    Returns:
        str: The cleaned and processed document as a single string.
    """
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    
    return [" ".join(final_token)]

In [14]:
abstract = "A fully differential calculation in perturbative quantum chromodynamics is presented for the production of massive photon pairs at hadron colliders. All next-to-leading order perturbative contributions from quark-antiquark, gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as all-orders resummation of initial-state gluon radiation valid at next-to-next-to-leading logarithmic accuracy. The region of phase space is specified in which the calculation is most reliable. Good agreement is demonstrated with data from the Fermilab Tevatron, and predictions are made for more detailed tests with CDF and DO data. Predictions are shown for distributions of diphoton pairs produced at the energy of the Large Hadron Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs boson are contrasted with those produced from QCD processes at the LHC, showing that enhanced sensitivity to the signal can be obtained with judicious selection of events. "

abstract_preprocessed = preprocessAbstracts(abstract)

In [18]:
_test = vectorizer.transform(abstract_preprocessed).toarray()
_test

array([[0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# def music_recomendation(dataset: pd.DataFrame, song_df: pd.DataFrame):
#     numerical_features_dataset = dataset[SONG_FEATURES]
#     numerical_features_song = song_df[SONG_FEATURES]
    
#     similarity = cosine_similarity(numerical_features_dataset, numerical_features_song)
#     similarity_df = pd.DataFrame(similarity, columns=['cosine_similarity'])
    
#     results = pd.concat([dataset, similarity_df], axis=1)
    
#     return results.sort_values('cosine_similarity', ascending=False).reset_index(drop=True).head()

In [62]:
tfidf_df[tfidf_features].to_array()

KeyboardInterrupt: 

In [59]:
similarity = cosine_similarity(tfidf_vectors, _test)

KeyError: "['00', '000', '0000002', '0000008', '0000010', '0000013', '000002', '00001', '00001887227319', '00002', '000085', '0001', '0002', '000201', '00023', '00026', '0003', '000301c', '0003184', '00034', '00036h', '0004', '0004016', '0004203', '0004222', '0005', '00057', '0007', '00079248', '0008', '00080', '00084', '0009', '000times', '001', '0010', '00105', '0011', '00116', '0012', '0012170', '00124', '0013', '00133', '00136', '0014', '00148', '0016257', '0017', '0019', '00197686', '002', '0021', '00226', '0023', '0024', '0029', '002coo_3', '003', '0031', '0033', '0035', '0038', '004', '00411', '0044', '0047', '0049', '00491', '004r', '005', '0050', '00513v1', '0052', '0055', '0056', '0057', '0059', '006', '0060', '00619', '0062', '0068', '00680', '007', '00713', '0073', '00759', '0076', '00788', '008', '0087', '008_', '009', '0093', '00938', '0094', '0095', '00_m', '01', '010', '010102', '010143', '010222', '0106554', '0107770', '010824', '0109', '011', '011013', '011032', '0115

In [ ]:
def inference(tfidf_df: pd.DataFrame, abstract: str):
    processed_abstract = preprocessAbstracts(abstract)
    abstract_vector = vectorizer.fit_transform(processed_abstract)
    
    
    return ""